In [1]:
%matplotlib inline

In [3]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.Conv1 = nn.Conv2d(1, 6, 5)
        self.Conv2 = nn.Conv2d(6, 16, 5)
        
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.Conv1(x)), (2,2))
        
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.Conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        
        
        
net = Net()
print(net)       

Net(
  (Conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (Conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [14]:
params = list(net.parameters())
print(len(params))

for i in range(10):
    print(i,'th: ', params[i].size())  # weight

10
0 th:  torch.Size([6, 1, 5, 5])
1 th:  torch.Size([6])
2 th:  torch.Size([16, 6, 5, 5])
3 th:  torch.Size([16])
4 th:  torch.Size([120, 400])
5 th:  torch.Size([120])
6 th:  torch.Size([84, 120])
7 th:  torch.Size([84])
8 th:  torch.Size([10, 84])
9 th:  torch.Size([10])


In [15]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

Variable containing:
1.00000e-02 *
  2.6498 -8.5693  4.8735 -5.4362 -2.9450  5.4403 -6.0944  8.8475  8.5076 -6.0278
[torch.FloatTensor of size 1x10]



In [16]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [17]:
output = net(input)
target = Variable(torch.arange(1, 11))  # a dummy target, for example
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.4225
[torch.FloatTensor of size 1]



In [18]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [20]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.Conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.Conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
 0.0295
-0.0834
 0.0992
 0.0084
 0.1248
-0.1165
[torch.FloatTensor of size 6]



In [21]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update